In [20]:
import os
import math

import matplotlib.pyplot as plt
import pandas as pd

CHR = {
    "chrI":    1,
    "chrII":   2,
    "chrIII":  3,
    "chrIV":   4,
    "chrV":    5,
    "chrVI":   6,
    "chrVII":  7,
    "chrVIII": 8,
    "chrIX":   9,
    "chrX":    10,
    "chrXI":   11,
    "chrXII":  12,
    "chrXIII": 13,
    "chrXIV":  14,
    "chrXV":   15,
    "chrXVI":  16,
    "chrM":    1000
}


class FrequencyAndMappingDataExtracted:
    def __init__(self, file_name, save_path):
        self.file_name = file_name
        self.save_path = save_path
        self.columns_name = ['chr', 'strand', 't5', 't3', 'ypd', 'gal', 'type', 'name']

    def read_bin_data(self):
        f_ori = open(self.file_name, 'r')
        count = 0

        data_list = []
        for line in f_ori.readlines():
            if count % 1000 == 0:
                print(count)
            # if count == 100:
            #     break
            line_list = line.split()
            res = line_list[:6]
            temp = ""
            for idx, ele in enumerate(line_list[6:]):
                if idx == len(line_list) - 7:
                    res.append(temp.rstrip())
                    res.append(ele)
                else:
                    temp = temp + ele + " "
            data_list.append(res)
            count += 1
        f_ori.close()
        data = pd.DataFrame(data_list, index=None, columns=self.columns_name)
        data.columns = data.loc[0]
        data = data[1:]
        data.to_csv(self.save_path + os.path.sep + "data.csv", index=None)

    def get_max_ypd(self, file_name):
        data_all = pd.read_csv(file_name)
        data_all.dropna(axis=0, how='any', inplace=True)

        genes = list(set(data_all["name"]))
        print(len(genes))
        result = []
        count = 0

        for gene in genes:
            if count % 500 == 0:
                print("count: ", count)
            max_ypd = data_all[(data_all["name"] == gene) & (data_all["type"] == "Covering one intact ORF")]
            if max_ypd.empty is False:
                max_ypd = max_ypd[max_ypd["ypd"] == max_ypd["ypd"].max()]
                result.append(max_ypd.values.tolist()[0])
            count += 1

        result_unsort = pd.DataFrame(result, columns=self.columns_name)
        result_unsort = result_unsort.groupby("chr")

        result_sort = pd.DataFrame(index=None, columns=self.columns_name)
        for name, group in result_unsort:
            group_sort = group.sort_values(by="t5")
            result_sort = result_sort.append(group_sort, ignore_index=True)
        print("result shape:", result_sort.shape)
        result_sort.to_csv(self.save_path + os.path.sep + "genes002.csv", index=None)


class Mapping:
    def __init__(self, file_path_gene, file_name_genes, save_path):
        self.file_path_gene = file_path_gene
        self.file_name_genes = file_name_genes
        self.save_path = save_path
        self.length_gene = 1000

    def plot_gene(self):
        f1 = pd.read_csv(self.file_path_gene)
        f2 = pd.read_csv(self.file_name_genes)
        f2_part = f2[f2["Chr"] == 1]
        plt.figure(figsize=(20, 5))
        plt.plot(f1["start"], f1["val"])
        # plt.scatter(f1["start"], f1["size"])
        # plt.scatter(data["start"], data["size"], s=0.2)
        # for start in f2_part["t5"]:
        #     plt.axvline(x=start, c="r", linestyle='dotted')
        # for end in f2_part["t3"]:
        #     plt.axvline(x=end, c="b", linestyle='dashed')
        plt.xlim()
        plt.ylim()
        plt.savefig(self.save_path + os.path.sep + self.file_path_gene[-8:-4] + ".png", dpi=90, bbox_inches='tight')
        plt.close()

    def get_array_1000_row(self):
        genes = pd.read_csv(self.file_name_genes)
        count = 0
        file_name_gene = ""
        genes_arr = []
        for index, row in genes.iterrows():
            if count % 100 == 0:
                print("count:", count)
            if file_name_gene[:-4] != str(row["chr"]):
                file_name_gene = str(row["chr"]) + ".csv"
                gene = pd.read_csv(self.file_path_gene + os.sep + file_name_gene)

            start = min(row["t3"], row["t5"])
            end   = max(row["t3"], row["t5"]) + 1
            if abs(start - end) < self.length_gene or row["ypd"] == 0:
                continue
            gene_arr = gene.iloc[start: end, [1]]
            gene_arr = gene_arr["size"].tolist()
            if row["t3"] < row["t5"]:
                gene_arr = gene_arr[::-1]
            gene_arr = gene_arr[:self.length_gene]
            genes_arr.append(row.tolist() + gene_arr)
            count += 1
        genes_arr_pd = pd.DataFrame(genes_arr)
        genes_arr_pd.to_csv(self.save_path + os.path.sep + "genes_arr_pd.csv", index=None)

    def get_array_1000_row_plus_1(self, t3, t5):
        genes = pd.read_csv(self.file_name_genes)
        count = 0
        file_name_gene = ""
        genes_arr = []
        genes[t5] = genes["-1 nucleosome"]
        genes[t3] = genes["ORF End"] + abs(genes["-1 nucleosome"] - genes["ORF Start"])
        print(genes.head())

        for index, row in genes.iterrows():
            if count % 100 == 0:
                print("count:", count)
            chr_name_mapping = str(CHR[row["Chr"]])
            if file_name_gene[:-4] != chr_name_mapping:
                file_name_gene = chr_name_mapping + ".csv"
                gene = pd.read_csv(self.file_path_gene + os.sep + file_name_gene)

            start = min(row[t3], row[t5])
            end   = max(row[t3], row[t5]) + 1
            if abs(start - end) < self.length_gene:
                continue
            gene_arr = gene.iloc[start: end, [1]]
            gene_arr = gene_arr["size"].tolist()
            if row[t3] < row[t5]:
                gene_arr = gene_arr[::-1]
            gene_arr = gene_arr[:self.length_gene]
            genes_arr.append(row.tolist() + gene_arr)
            # print(len(genes_arr), len(genes_arr[0]))
            count += 1
        genes_arr_pd = pd.DataFrame(genes_arr)
        genes_arr_pd.to_csv(self.save_path + os.path.sep + "genes_arr_pd_plus_1.csv", index=None)

        print(genes_arr_pd.shape)
        # print(genes_arr_pd)


def run_preprocess():
    file_name = r'D:\code\Final_project_data_science\data\S2_tcd_mTIFAnno.txt'
    save_path = r'D:\code\Final_project_data_science\result'
    extracted = FrequencyAndMappingDataExtracted(file_name, save_path)

    # txt to csv
    # prepro.read_bin_data()

    data_csv_file_name = r'D:\code\Final_project_data_science\data\data.csv'
    extracted.get_max_ypd(data_csv_file_name)


def run_process():
    file_path_gene = r'..\data\genes_all'
    file_name_genes = r'..\data\13059_2018_1398_MOESM2_ESM_using.csv'
    save_path = r"..\result"
    mapping = Mapping(file_path_gene, file_name_genes, save_path)
    # process.plot_gene()
    mapping.get_array_1000_row_plus_1(t3="t3", t5="t5")


In [ ]:
run_preprocess()

In [22]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns

from sklearn import preprocessing
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN


class Cluster:
    def __init__(self, data, save_path=r"..\result"):
        self.data = data
        self.length_of_gene = 1000
        self.step = 10
        self.score = 0
        self.save_path = save_path

    def plot(self):
        plt.figure(figsize=(10, 3))
        plt.plot(range(0, self.length_of_gene), self.data.loc[0])
        plt.xlabel("BP")
        plt.ylabel("Frequency")
        plt.show()
        plt.close()

    def normalization(self):
        self.data = self.data.rolling(window=self.step, win_type='gaussian', min_periods=1, axis='columns').mean(std=10)
        self.plot()
        self.data = self.data.T
        my_scaler = preprocessing.StandardScaler().fit(self.data)
        self.data = my_scaler.transform(self.data)
        self.data = self.data.T
        self.data = pd.DataFrame(self.data)

        self.plot()

        return self.data

    def cluster_fit(self, method):
        """Benchmark to evaluate the KMeans initialization methods.

        Parameters
        ----------
        method : KMeans instance
            A :class:`~sklearn.cluster.KMeans` instance with the initialization
            already set.
        """
        estimator = method.fit_predict(self.data)
        score = metrics.silhouette_score(self.data, estimator)
        self.score = score
        # self.plot()
        return estimator

    def heat_map(self):
        bounds = np.linspace(-1.5, 1.5, 15)
        plt.imshow(self.data, norm=colors.BoundaryNorm(boundaries=bounds, ncolors=256), origin="lower", aspect=0.5)
        plt.colorbar()
        plt.show()

    def get_dbscan_eps(self):
        data_np = np.array(self.data)
        print(data_np.shape)
        # data_np = data_np[0: 100, :20]

        def select_MinPts(data, k):
            k_dist = []
            for i in range(data.shape[0]):
                dist = (((data[i] - data) ** 2).sum(axis=1) ** 0.5)
                dist.sort()
                k_dist.append(dist[k])
                print("i: ", i, dist[k])
            return np.array(k_dist)

        k = data_np.shape[1] * 2 - 1
        k_dist = select_MinPts(data_np, k)
        k_dist.sort()
        k_dist_pd = pd.DataFrame({"k:dist": k_dist})
        k_dist_pd.to_csv(self.save_path + os.path.sep + "DBSCAN_k_dist.csv", index=None)

        plt.figure()
        plt.plot(np.arange(k_dist.shape[0]), k_dist[::-1])
        plt.show()
        plt.figure()
        plt.scatter(np.arange(k_dist.shape[0]), k_dist[::-1])
        plt.show()


class ResultAnalysis:
    def __init__(self, result_data, save_path, method):
        self.result_data = result_data
        self.save_path = save_path
        self.method = method
        grouped_genes = self.result_data.groupby("predict labels")
        self.mean_of_grouped_genes = grouped_genes.agg("mean")
        self.length_gene = 1000

        # print(type(grouped_genes.groups))

        print(80 * "*")
        self.groups = {}
        for key, item in grouped_genes.groups.items():
            print("{0} : {1}".format(key, len(item)))
            self.groups[key] = len(item)
        print(80 * "*")
        # print(grouped_genes.groups)

    def plot_means(self):
        font_size = 16
        plt.figure()
        for idx, row in self.mean_of_grouped_genes.iterrows():
            # row = gaussian_filter1d(row, sigma=5)
            plt.plot(range(0, self.length_gene), row, label=str(idx) + "-->" + str(self.groups[idx]))
            plt.legend()

        # plt.title(str(idx))
        plt.xlabel("BP", fontsize=font_size)
        plt.ylabel("Means Normalised Reads", fontsize=font_size)

        plt.savefig(self.save_path + os.path.sep + self.method + "_0729.png",
                    dpi=180,
                    bbox_inches='tight')
        plt.close()


def get_kmeans_k(cluster_method="DBSCAN", feature_start=8, file_name=r'..\data\genes_arr_pd.csv'):

    data_ori = pd.read_csv(file_name)
    data = data_ori.iloc[:, feature_start:]

    cluster = Cluster(data)
    cluster.normalization()
    scores = []
    for n_clusters in range(2, 12):
        print("i: ", n_clusters)
        cluster_methods = KMeans(init="k-means++", n_clusters=n_clusters, random_state=0)
        cluster.cluster_fit(method=cluster_methods[cluster_method])
        scores.append(cluster.score)

    fontsize = 15
    plt.figure()
    plt.plot(range(2, 12), scores)
    plt.scatter(range(2, 12), scores, c="r")
    plt.xlabel("K", fontsize=fontsize)
    plt.ylabel("Silhouette score", fontsize=fontsize)
    plt.show()
    plt.close()


def dbscan_eps_and_min_samples(feature_start=8,
                               file_name=r'..\data\genes_arr_pd.csv'):
    save_path = r"..\result"
    data_ori = pd.read_csv(file_name)
    data = data_ori.iloc[:, feature_start:]

    cluster = Cluster(data=data, save_path=save_path)
    cluster.normalization()
    # cluster.get_dbscan_eps()

    eps_begin = 28
    eps_end = 32
    mim_samples_begin = 98
    min_samples_end = 120
    eps = np.arange(eps_begin, eps_end, 1)
    min_samples = np.arange(mim_samples_begin, min_samples_end, 2)
    results = [[0 for _ in range(5)]]

    for i in eps:
        for j in min_samples:
            print(i, j)
            try:
                cluster_method = DBSCAN(eps=i, min_samples=j)
                predict_labels = cluster.cluster_fit(method=cluster_method)
                score = metrics.silhouette_score(data, predict_labels)  # 轮廓系数评价聚类的好坏，值越大越好
                raito = len(predict_labels[predict_labels[:] == -1]) / len(predict_labels)  # 计算噪声点个数占总数的比例
                n_clusters = len(set(predict_labels)) - (1 if -1 in predict_labels else 0)  # 获取分簇的数目
                results.append([i, j, score, raito, n_clusters])
                print("gird: ", [i, j, score, raito, n_clusters])
            except:
                continue
    results = pd.DataFrame(results)
    results.columns = ['eps', 'min_samples', 'score', 'raito', 'n_clusters']

    sns.relplot(x="eps", y="min_samples", size='score', data=results, hue="n_clusters")
    plt.xlim([23, 35])
    plt.ylim([93, 106])
    plt.show()
    sns.relplot(x="eps", y="min_samples", size='raito', data=results, hue="n_clusters")
    plt.xlim([23, 35])
    plt.ylim([93, 106])
    plt.show()


def run_cluster(cluster_method="k-mean++", feature_start=8, file_name=r'..\data\genes_arr_pd.csv'):
    n_digits = 6
    save_path = r"..\result"
    data_ori = pd.read_csv(file_name)
    data = data_ori.iloc[:, feature_start:]

    cluster = Cluster(data)
    cluster.normalization()
    (n_samples, n_features) = data.shape

    print(f"# digits: {n_digits}; # samples: {n_samples}; # features {n_features}")
    print(82 * "_")
    cluster_methods = {
        "k-means++": KMeans(init="k-means++", n_clusters=n_digits, random_state=0),
        "DBSCAN": DBSCAN(eps=30, min_samples=100)
    }
    predict_labels = cluster.cluster_fit(method=cluster_methods[cluster_method])
    print("score: ", cluster.score)

    method = cluster_method + str(n_digits)

    data["predict labels"] = predict_labels
    predict_labels_data = data_ori.iloc[:, :feature_start + 1]
    predict_labels_data["predict labels"] = predict_labels

    # predict_labels_data.to_csv(save_path + os.path.sep + cluster_method + "_predict_result.csv", index=None)

    print(data.head())

    result_analysis = ResultAnalysis(data, save_path, method)
    result_analysis.plot_means()

    print(82 * "_")

In [ ]:
dbscan_eps_and_min_samples(feature_start=8, 
                           file_name=r'D:\code\Final_project_data_science\data\genes_arr_pd.csv')
